In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [97]:
data = pd.read_csv('wall-robot-nav.csv').sample(frac=1)
X = data.loc[:, 'V1':'V24']
X = X / np.array(X).max(axis=0)
y = data['Class']
train_size = math.ceil(X.shape[0]*0.8)
test_size = X.shape[0] - train_size
X_train = X.head(train_size)
y_train = y.head(train_size)
X_test = X.tail(test_size)
y_test = y.tail(test_size)

In [98]:

def distance_euclid(x, xi):
    return np.sqrt((np.square(x-xi)).sum(axis=1))


def uniform_kernel(x):
    new_x = x
    new_x[new_x<=1] = 0.5
    new_x[new_x>1] = 0
    return new_x

class KernelRegressionClassifier:

    distance_functions = {
        'euclid': distance_euclid
    }

    kernels = {
        'uniform': uniform_kernel,
    }

    def __init__(self, X, y, h, k=10, kernel_name='uniform', distance_func_name='euclid'):
        self.k = k
        self.h = h
        self.kernel_name = kernel_name
        self.distance_func_name = distance_func_name
        self.X = X
        self.y = np.array(y)
        self.kernel = self.kernels[self.kernel_name]
        self.distance_func = self.distance_functions[self.distance_func_name]

    def predict(self, x):
        X_distance = np.apply_along_axis(self.distance_func, 1, x, self.X)
        closest_indexes_matrix = X_distance.argsort()
        X_smoothed = self.kernel(X_distance/self.h)
        result = []
        for i in range(len(X_smoothed)):
            row = X_smoothed[i]
            indexes_of_clossest = closest_indexes_matrix[i]
            index_of_clossest = indexes_of_clossest[:self.k]
            alg = (row[index_of_clossest] * self.y[index_of_clossest]).sum() / (row[index_of_clossest].sum() or 1)
            result.append(math.ceil(alg))
        return result

In [99]:
clf = KernelRegressionClassifier(X_train, y_train, h=10, k=10 )
print('Accuracy: ', (clf.predict(X_test) == y_test).astype('int8').sum()/test_size)

Accuracy:  0.615032080659945
